# Test memory-allocations IRKGL16 with adaptive step size

## Loading packages and functions

In [1]:
using LinearAlgebra, Plots
using OrdinaryDiffEq
using IRKGaussLegendre



## Definition of the N-body problem

In Nbody.jl below, the following functions are defined: NbodyEnergy(u,Gm), NbodyODE!(du,u,Gm,t), and NbodyODE1!(du,u,Gm,t), where

\begin{equation*}
u = 
\left(\begin{matrix}
q_1  & v_1\\
\vdots & \vdots \\
q_N  & v_N
\end{matrix}
\right) \in \mathbb{R}^{2 \times 3\times N}, \quad
Gm = (G\,  m_1, \ldots,G\,  m_N) \in \mathbb{R}^N.
\end{equation*}

The energy, as a function of the positions $q_1,\ldots,q_N \in \mathbb{R}^3$ and the velocities $v_1,\ldots,v_N \in \mathbb{R}^3$ of the $N$ bodies is:
\begin{equation*}
\begin{split}
\mathrm{Energy}(q_1,\ldots,q_N,v_1,\ldots,v_N) = 
\frac12 \sum_{i=1}^{N} m_i \, \|v_i\|^2
- G \sum_{1\leq i < j \leq N} \frac{m_i m_j}{\|q_i-q_j\|}.
\end{split}
\end{equation*}


The ODE system of the N-body problem,
as a function of the positions $q_1,\ldots,q_N \in \mathbb{R}^3$ and the velocities $v_1,\ldots,v_N \in \mathbb{R}^3$ of the $N$ bodies is:
\begin{equation*}
\begin{split}
\frac{d}{dt} q_i &= v_i, \\
\frac{d}{dt} v_i &= G\, 
\sum_{j \neq i} \frac{m_j}{\|q_j-q_i\|^3}\,
(q_j-q_i).
\end{split}
\end{equation*}
This system of ODEs can be writen in compact form as
\begin{equation*}
\frac{d u}{dt} = f(t,u,Gm)
\end{equation*}




In [3]:
PATH_ODES="../ODEProblems/"

include(string(PATH_ODES,"Initial6Body.jl"))
include(string(PATH_ODES,"Nbody.jl"));

### Initial value problem: 6-body problem (outer solar system)

We consider $N=6$ bodies of the outer solar system: the Sun, Jupiter, Saturn, Uranus, Neptune and Pluto.
The initial values $u_{00}$ are taken from DE430, Julian day (TDB) 2440400.5 (June 28, 1969). 

In [4]:
u0, Gm, bodylist = Initial6Body(Float64)
u0_B, Gm_B, bodylist = Initial6Body(BigFloat)

q0=u0[:,:,1]
v0=u0[:,:,2]
dim=length(size(u0))

N = length(Gm)

show(bodylist)
E0=NbodyEnergy(u0,Gm)

["Sun" "Jupiter" "Saturn" "Uranus" "Neptune" "Pluto"]

-9.522620605966966e-12

##### <a href="#top">Back to the top</a>

## Memory allocations

In [5]:
t0 = 0.
dt = 500.  # 500.

tF = 100*dt
tF2 = 2*tF
tF4 = 4*tF

prob = ODEProblem(NbodyODE!, u0, (t0,tF), Gm)
prob2 = ODEProblem(NbodyODE!, u0, (t0,tF2), Gm)
prob4 = ODEProblem(NbodyODE!, u0, (t0,tF4), Gm);

### Sequential version

#### IRKGL_seq

In [6]:
alg_seq=IRKGL16(second_order_ode=false, simd=false)

solx=solve(prob, alg_seq,  adaptive=false, dt=dt, save_everystep=false)
solx2=solve(prob2,alg_seq, adaptive=false, dt=dt, save_everystep=false)
solx4=solve(prob4,alg_seq, adaptive=false, dt=dt,save_everystep=false);


In [7]:
println(solx.retcode,",tspan=",prob.tspan,",steps=",solx.destats.naccept,",length(solx.t)=",length(solx.t))
@time solve(prob,alg_seq, dt=dt,adaptive=false, save_everystep=false)
@time solve(prob,alg_seq, dt=dt,adaptive=false, save_everystep=false);

Success,tspan=(0.0, 50000.0),steps=100,length(solx.t)=2
  0.063313 seconds (83.10 k allocations: 5.759 MiB, 27.71% gc time, 87.41% compilation time)
  0.007028 seconds (206 allocations: 33.391 KiB)


In [8]:
println(solx2.retcode,",tspan=",prob2.tspan,",steps=",solx2.destats.naccept,",length(solx2.t)=",length(solx2.t))
@time solve(prob2,alg_seq,dt=dt, adaptive=false, save_everystep=false);

Success,tspan=(0.0, 100000.0),steps=200,length(solx2.t)=2
  0.013649 seconds (306 allocations: 41.203 KiB)


In [9]:
println(solx4.retcode,",tspan=",prob4.tspan,",steps=",solx4.destats.naccept,",length(solx4.t)=",length(solx4.t))
@time solve(prob4,alg_seq,dt=dt,  adaptive=false, save_everystep=false);

Success,tspan=(0.0, 200000.0),steps=400,length(solx4.t)=2
  0.028310 seconds (506 allocations: 56.828 KiB)


#### IRKNGL_seq

In [10]:
alg_seq=IRKGL16(second_order_ode=true, simd=false)

solx=solve(prob, alg_seq,  adaptive=false, dt=dt, save_everystep=false)
solx2=solve(prob2,alg_seq, adaptive=false, dt=dt, save_everystep=false)
solx4=solve(prob4,alg_seq, adaptive=false, dt=dt, save_everystep=false);

In [11]:
println(solx.retcode,",tspan=",prob.tspan,",steps=",solx.destats.naccept,",length(solx.t)=",length(solx.t))
@time solve(prob,alg_seq, dt=dt,adaptive=false, save_everystep=false)
@time solve(prob,alg_seq, dt=dt,adaptive=false, save_everystep=false);

Success,tspan=(0.0, 50000.0),steps=100,length(solx.t)=2
  0.023745 seconds (26.61 k allocations: 1.873 MiB, 82.62% compilation time)
  0.003314 seconds (206 allocations: 33.391 KiB)


In [12]:
println(solx2.retcode,",tspan=",prob2.tspan,",steps=",solx2.destats.naccept,",length(solx2.t)=",length(solx2.t))
@time solve(prob2,alg_seq,dt=dt, adaptive=false, save_everystep=false);

Success,tspan=(0.0, 100000.0),steps=200,length(solx2.t)=2
  0.006458 seconds (306 allocations: 41.203 KiB)


In [13]:
println(solx4.retcode,",tspan=",prob4.tspan,",steps=",solx4.destats.naccept,",length(solx4.t)=",length(solx4.t))
@time solve(prob4,alg_seq,dt=dt,  adaptive=false, save_everystep=false);

Success,tspan=(0.0, 200000.0),steps=400,length(solx4.t)=2
  0.012367 seconds (506 allocations: 56.828 KiB)


### Simd version

#### IRKGL_simd

In [14]:
alg_simd=IRKGL16(second_order_ode=false, simd=true)


solx=solve(prob, alg_simd,  adaptive=false, dt=dt, save_everystep=false)
solx2=solve(prob2,alg_simd, adaptive=false, dt=dt, save_everystep=false)
solx4=solve(prob4,alg_simd, adaptive=false, dt=dt, save_everystep=false);


In [15]:
println(solx.retcode,",tspan=",prob.tspan,",steps=",solx.destats.naccept,",length(solx.t)=",length(solx.t))
@time solve(prob,alg_simd,dt=dt,adaptive=false, save_everystep=false)
@time solve(prob,alg_simd,dt=dt,adaptive=false, save_everystep=false);

Success,tspan=(0.0, 50000.0),steps=100,length(solx.t)=2
  0.019943 seconds (26.59 k allocations: 1.894 MiB, 89.87% compilation time)
  0.001559 seconds (182 allocations: 54.281 KiB)


In [16]:
println(solx2.retcode,",tspan=",prob2.tspan,",steps=",solx2.destats.naccept,",length(solx2.t)=",length(solx2.t))
@time solve(prob2,alg_simd,dt=dt, adaptive=false, save_everystep=false);

Success,tspan=(0.0, 100000.0),steps=200,length(solx2.t)=2
  0.002749 seconds (282 allocations: 87.094 KiB)


In [17]:
println(solx4.retcode,",tspan=",prob4.tspan,",steps=",solx4.destats.naccept,",length(solx4.t)=",length(solx4.t))
@time solve(prob4,alg_simd,dt=dt,  adaptive=false, save_everystep=false);

Success,tspan=(0.0, 200000.0),steps=400,length(solx4.t)=2
  0.005428 seconds (482 allocations: 152.719 KiB)


#### IRKNGL_simd

In [18]:
alg_simd=IRKGL16(second_order_ode=true, simd=true)

solx=solve(prob, alg_simd,  adaptive=false, dt=dt, save_everystep=false)
solx2=solve(prob2,alg_simd, adaptive=false, dt=dt, save_everystep=false)
solx4=solve(prob4,alg_simd, adaptive=false, dt=dt, save_everystep=false);

In [19]:
println(solx.retcode,",tspan=",prob.tspan,",steps=",solx.destats.naccept,",length(solx.t)=",length(solx.t))
@time solve(prob,alg_simd,dt=dt,adaptive=false, save_everystep=false)
@time solve(prob,alg_simd,dt=dt,adaptive=false, save_everystep=false);

Success,tspan=(0.0, 50000.0),steps=100,length(solx.t)=2
  0.019765 seconds (26.59 k allocations: 1.893 MiB, 95.63% compilation time)
  0.000821 seconds (182 allocations: 54.281 KiB)


In [20]:
println(solx2.retcode,",tspan=",prob2.tspan,",steps=",solx2.destats.naccept,",length(solx2.t)=",length(solx2.t))
@time solve(prob2,alg_simd,dt=dt, adaptive=false, save_everystep=false);

Success,tspan=(0.0, 100000.0),steps=200,length(solx2.t)=2
  0.001536 seconds (282 allocations: 87.094 KiB)


In [21]:
println(solx4.retcode,",tspan=",prob4.tspan,",steps=",solx4.destats.naccept,",length(solx4.t)=",length(solx4.t))
@time solve(prob4,alg_simd,dt=dt,  adaptive=false, save_everystep=false);

Success,tspan=(0.0, 200000.0),steps=400,length(solx4.t)=2
  0.003290 seconds (482 allocations: 152.719 KiB)


### Vern9

In [22]:
sol_Vern =solve(prob,Vern9(), dt=dt, save_everystep=false)
sol_Vern2 =solve(prob2,Vern9(), dt=dt, save_everystep=false)
sol_Vern4 =solve(prob4,Vern9(), dt=dt, save_everystep=false);


In [23]:
println(sol_Vern.retcode,",tspan=",prob.tspan,",steps=",prob.tspan[2]/dt,",length(sol_Vern.t)=",length(sol_Vern.t))
@time solve(prob,Vern9(),dt=dt, save_everystep=false);

Success,tspan=(0.0, 50000.0),steps=100.0,length(sol_Vern.t)=2
  0.000165 seconds (54 allocations: 10.016 KiB)


In [24]:
println(sol_Vern2.retcode,",tspan=",prob2.tspan,",steps=",prob2.tspan[2]/dt,",length(sol_Vern2.t)=",length(sol_Vern2.t))
@time solve(prob2,Vern9(),dt=dt, save_everystep=false);

Success,tspan=(0.0, 100000.0),steps=200.0,length(sol_Vern2.t)=2
  0.000360 seconds (54 allocations: 10.016 KiB)


In [25]:
println(sol_Vern4.retcode,",tspan=",prob4.tspan,",steps=",prob4.tspan[2]/dt,",length(sol_Vern4.t)=",length(sol_Vern4.t))
@time solve(prob4,Vern9(),dt=dt, save_everystep=false);

Success,tspan=(0.0, 200000.0),steps=400.0,length(sol_Vern4.t)=2
  0.000688 seconds (54 allocations: 10.016 KiB)


##### <a href="#top">Back to the top</a>



## Error in Energy

In [26]:
alg_seq=IRKGL16(second_order_ode=false, simd=false)
alg_simd=IRKGL16(second_order_ode=false, simd=true)

sol1=solve(prob,alg_seq,dt=dt/2, adaptive=false)
println(sol1.retcode,",length(sol1.t)=",length(sol1.t))

sol2=solve(prob,alg_simd,dt=dt/2, adaptive=false)
println(sol2.retcode,",length(sol2.t)=",length(sol2.t))

alg_seq=IRKGL16(second_order_ode=true, simd=false)
alg_simd=IRKGL16(second_order_ode=true, simd=true)

sol3=solve(prob,alg_seq,dt=dt/2, adaptive=false)
println(sol3.retcode,",length(sol3.t)=",length(sol3.t))

sol4=solve(prob,alg_simd,dt=dt/2, adaptive=false)
println(sol4.retcode,",length(sol4.t)=",length(sol4.t))

Success,length(sol1.t)=201
Success,length(sol2.t)=201
Success,length(sol3.t)=201
Success,length(sol4.t)=201


In [27]:
E0=NbodyEnergy(u0_B, Gm_B)
year=365.25
yrange=(1e-18,1e-14)
ΔE1 = map(x->NbodyEnergy(BigFloat.(x),Gm_B), sol1.u)./E0.-1
ΔE2 = map(x->NbodyEnergy(BigFloat.(x),Gm_B), sol2.u)./E0.-1
ΔE3 = map(x->NbodyEnergy(BigFloat.(x),Gm_B), sol3.u)./E0.-1
ΔE4 = map(x->NbodyEnergy(BigFloat.(x),Gm_B), sol4.u)./E0.-1


pl1=plot(title="Error in energy (IRK)",xlabel="t (years)", ylabel="log10(ΔE/E0)", 
     yscale=:log10, ylims=yrange, legend=:bottom)

plot!(pl1, sol1.t./year,abs.(ΔE1),label="IRKGL_gen", color="orange")
plot!(pl1, sol2.t./year,abs.(ΔE2),label="IRKGL_simd", color="red")

pl2=plot(title="Error in energy (IRKN)",xlabel="t (years)", ylabel="log10(ΔE/E0)", 
     yscale=:log10, ylims=yrange, legend=:bottom)

plot!(pl2, sol3.t./year,abs.(ΔE3),label="IRKNGL_gen", color="orange")
plot!(pl2, sol4.t./year,abs.(ΔE4),label="IRKNGL_simd", color="red")

plot(pl1, pl2, layout=(1,2), size=(900,300))